<a href="https://colab.research.google.com/github/amgothhrithik/Go_emotion_label_Classification-using-Roberta-Model/blob/main/go_emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ast
import numpy as np
train = pd.read_csv("corrected_vad_train.csv")
train["labels"] = train["labels"].apply(ast.literal_eval)
train["VAD"] = train["VAD"].apply(ast.literal_eval)


train["VAD"] = train["VAD"].apply(lambda x: [np.float32(num) for num in x])

test = pd.read_csv("corrected_vad_test.csv")
test["labels"] = test["labels"].apply(ast.literal_eval)
test["VAD"] = test["VAD"].apply(ast.literal_eval)
test["VAD"] = test["VAD"].apply(lambda x: [np.float32(num) for num in x])
val = pd.read_csv("corrected_vad_val.csv")
val["labels"] = val["labels"].apply(ast.literal_eval)
val["VAD"] = val["VAD"].apply(ast.literal_eval)
val["VAD"] = val["VAD"].apply(lambda x: [np.float32(num) for num in x])
emotion_labels_pd = pd.read_csv("emotion_labels.csv")


# Check the result
print(train.head())
print(val.head())
print(test.head())
print(emotion_labels_pd.head())

                                                text      labels  \
0  I crave the warmth of your touch, the way your...     [8, 18]   
1  Every time you walk into the room, I feel an u...     [8, 13]   
2  I want to know every version of you, from the ...  [8, 7, 18]   
3  There’s a hunger in me that only your presence...         [8]   
4  I long to be the reason behind your smile, the...     [8, 17]   

                        VAD  
0     [0.598, 0.418, 0.561]  
1     [0.541, 0.433, 0.429]  
2      [0.584, 0.437, 0.54]  
3   [0.488, 0.4704, 0.4472]  
4  [0.4928, 0.2664, 0.3752]  
                                                text   labels  \
0  Is this in New Orleans?? I really feel like th...     [27]   
1  You know the answer man, you are programmed to...  [4, 27]   
2               I've never been this sad in my life!     [25]   
3  The economy is heavily controlled and subsidiz...  [4, 27]   
4  He could have easily taken a real camera from ...     [20]   

                    

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
print(type(train["labels"][0]))

<class 'list'>


In [ ]:
train.shape

(48339, 2)

In [ ]:
emotion_labels=list(emotion_labels_pd["emotions"])
print(emotion_labels)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
from collections import Counter
all_labels=[]

for labels in train["labels"]:
    all_labels.extend(labels)
label_counts = Counter(all_labels)
emotion_distribution = {}
for label, count in label_counts.items():
    emotion_distribution[emotion_labels[label]] = count


emotion_distribution = dict(sorted(emotion_distribution.items(), key=lambda item: item[1]))

print("emotions_distribution",emotion_distribution)




emotions_distribution {'disgust': 935, 'nervousness': 1016, 'embarrassment': 1030, 'desire': 1060, 'pride': 1062, 'excitement': 1068, 'caring': 1122, 'relief': 1160, 'grief': 1160, 'remorse': 1169, 'surprise': 1172, 'fear': 1228, 'realization': 1379, 'disappointment': 1387, 'confusion': 1419, 'sadness': 1610, 'anger': 1652, 'joy': 1659, 'optimism': 1815, 'disapproval': 2056, 'curiosity': 2283, 'love': 2397, 'amusement': 2497, 'annoyance': 2533, 'gratitude': 2732, 'approval': 3002, 'admiration': 4207, 'neutral': 14176}


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from transformers.utils import move_cache

# Migrate old cache (one-time operation)
move_cache()

0it [00:00, ?it/s]

In [ ]:
from transformers import RobertaTokenizer,BertTokenizer
import torch

from transformers import RobertaForSequenceClassification,BertForSequenceClassification
from transformers import TrainingArguments, Trainer


model="roberta-base"
tokenizer= RobertaTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
val_dataset = Dataset.from_pandas(val)
def processing_data(example_batch):
  tokens=tokenizer(example_batch["text"],padding="max_length",truncation=True,max_length=256,return_tensors="pt")

  total_labels=28
  batch_size=len(example_batch["text"])

  labels=torch.zeros((batch_size,total_labels),dtype=torch.float32)

  for idx, label_list in enumerate(example_batch["labels"]):
    for label in label_list:
      labels[idx][label]=1.0
  tokens["labels_ht"]=labels.tolist()
  return tokens
tokenized_train=train_dataset.map(processing_data,batched=True,remove_columns=["text","labels"])
tokenized_test=test_dataset.map(processing_data,batched=True,remove_columns=["text","labels"])
tokenized_val=val_dataset.map(processing_data,batched=True,remove_columns=["text","labels"])

Map:   0%|          | 0/48339 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

In [ ]:
tokenized_train = tokenized_train.rename_column("labels_ht", "labels")
tokenized_test= tokenized_test.rename_column("labels_ht", "labels")
tokenized_val= tokenized_val.rename_column("labels_ht", "labels")


tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



In [ ]:
print(tokenized_train )
print(type(tokenized_train[2]["labels"]))
print(tokenized_train[2]["labels"])

Dataset({
    features: ['VAD', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 48339
})
<class 'torch.Tensor'>
tensor([0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
model_bert = RobertaForSequenceClassification.from_pretrained(model, num_labels=28, problem_type="multi_label_classification")
model_bert.to(device)

from sklearn.metrics import f1_score, accuracy_score,precision_score, recall_score


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits).to(device)).cpu().numpy()
    threshould=0.3
    preds = (probs > threshould).astype(int)
    f1 = f1_score(labels, preds, average="weighted", zero_division=1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted", zero_division=1)
    recall = recall_score(labels, preds, average="weighted", zero_division=1)

    return {"f1": f1, "accuracy": accuracy, "precision": precision, "recall": recall}
# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data/go_emotion/output",  # Save locally
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Increase to 5
    weight_decay=0.01,
    learning_rate=2e-5,  # Lower learning rate
    lr_scheduler_type="cosine",  # Use cosine decay
    logging_dir="/content/drive/My Drive/data/go_emotion/logs",  # Save logs locally
    report_to="none",
    fp16=True,
)


from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from transformers import Trainer


# Define Trainer
trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train model
trainer.train()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
1,0.094700,0.087750,0.578208,0.435864,0.598245,0.598433
2,0.081600,0.082669,0.594282,0.460745,0.597588,0.617085
3,0.072400,0.082379,0.603732,0.452083,0.585533,0.633699


TrainOutput(global_step=9066, training_loss=0.0944803302292789, metrics={'train_runtime': 2071.9259, 'train_samples_per_second': 69.991, 'train_steps_per_second': 4.376, 'total_flos': 1.908224154333389e+16, 'train_loss': 0.0944803302292789, 'epoch': 3.0})

In [ ]:
# Evaluate model
trainer.evaluate(tokenized_test)

{'eval_loss': 0.08171425759792328,
 'eval_f1': 0.6074532964732495,
 'eval_accuracy': 0.4484982494932744,
 'eval_precision': 0.586950715308699,
 'eval_recall': 0.6405435297835361,
 'eval_runtime': 19.8198,
 'eval_samples_per_second': 273.817,
 'eval_steps_per_second': 17.155,
 'epoch': 3.0}

# Loading model

In [5]:
import torch
from transformers import RobertaForSequenceClassification,RobertaTokenizer
path="/content/drive/MyDrive/Data/go_emotion/checkpt/checkpt_2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model=RobertaForSequenceClassification.from_pretrained(path, local_files_only=True).to(device)

tokenizer= RobertaTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
import pandas as pd
emotion_labels_pd = pd.read_csv("/content/drive/My Drive/Data/go_emotion/data/emotion_labels.csv")
emotion_labels=list(emotion_labels_pd["emotions"])
print(emotion_labels)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [24]:

def predict_emotion(text, emotion_distribution, threshold=0.15):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.sigmoid(logits)  # Convert logits to probabilities
    predicted_labels = (probs > threshold).int().tolist()[0]  # Apply threshold

    # Get emotion names

    predicted_emotions = [emotion_distribution[i] for i, val in enumerate(predicted_labels) if val == 1]

    return predicted_emotions

# Example

sentences = [
    "Her heart felt like a heavy stone, and no matter how much she tried to hold back, tears kept slipping down her face.",
    "I can't believe you betrayed me like this! I feel so angry and hurt.",
    "This is absolutely amazing! I'm so thrilled and proud of you!",
    "I don't know how to face them after what happened. I'm so ashamed.",
    "That joke was so funny, I can't stop laughing!",
    "You are such an inspiration! I truly admire your strength.",
    "I'm really scared about tomorrow’s surgery. What if something goes wrong?",
    "I miss you so much. Life feels so empty without you here.",
    "Ugh, this traffic is the worst! So frustrating!",
    "Wow, I wasn't expecting this at all. I'm completely surprised!",
    "I feel so at peace when I sit by the ocean and listen to the waves."
]



for i in sentences:
  print(i.ljust(100),":",predict_emotion(i,emotion_labels))


Her heart felt like a heavy stone, and no matter how much she tried to hold back, tears kept slipping down her face. : ['grief', 'sadness']
I can't believe you betrayed me like this! I feel so angry and hurt.                                 : ['anger', 'annoyance']
This is absolutely amazing! I'm so thrilled and proud of you!                                        : ['admiration', 'excitement', 'pride']
I don't know how to face them after what happened. I'm so ashamed.                                   : ['embarrassment', 'grief', 'remorse']
That joke was so funny, I can't stop laughing!                                                       : ['amusement']
You are such an inspiration! I truly admire your strength.                                           : ['admiration']
I'm really scared about tomorrow’s surgery. What if something goes wrong?                            : ['fear', 'nervousness']
I miss you so much. Life feels so empty without you here.                                 